In [1]:
%cd '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect'

/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect


In [17]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from keras.layers.normalization import BatchNormalization
from augment.final.face_org import * 

Using TensorFlow backend.


In [10]:
image_folder_healthy = '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect/data/parsed/healthy'
image_folder_sick = '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect/data/parsed/sick'

files_healthy = os.listdir(image_folder_healthy)
files_sick = os.listdir(image_folder_sick)

train_data = []
test_data = []

sick = np.array([1,0])
healthy = np.array([0,1])

IMAGE_SIZE = 217
 
for f in files_healthy:
    full_path = image_folder_healthy + "/" + str(f)
    image = cv2.cvtColor(full_path, cv2.COLOR_BGR2GRAY)
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
    train_data.append([np.array(image), healthy])
for f in files_sick:
    full_path = image_folder_healthy + "/" + str(f)
    image = cv2.cvtColor(full_path, cv2.COLOR_BGR2GRAY)
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
    train_data.append([np.array(image), sick])

shuffle(train_data)
test_data = train_data[len(train_data)-5:]
train_data = train_data[:-5]

print('train_images shape:', np.array(train_images).shape)
print('test_images shape:', np.array(test_images).shape)


TypeError: Expected Ptr<cv::UMat> for argument 'src'

In [18]:
model = models.Sequential()

model.add(layers.Conv2D(IMAGE_SIZE, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(IMAGE_SIZE, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(int(IMAGE_SIZE/2), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(int(IMAGE_SIZE/2), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(int(IMAGE_SIZE/4), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(int(IMAGE_SIZE/4), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(int(IMAGE_SIZE/8), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(int(IMAGE_SIZE/8), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(int(IMAGE_SIZE/16), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(int(IMAGE_SIZE/16), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(217, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))


TypeError: The added layer must be an instance of class Layer. Found: <keras.layers.normalization.BatchNormalization object at 0x7fd8f0557d90>

In [ ]:
model.summary()

As you can see, our (4, 4, 64) outputs were flattened into vectors of shape (1024) before going through two Dense layers.

### Compile and train the model

In [23]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

ValueError: Error when checking input: expected conv2d_input to have shape (32, 32, 3) but got array with shape (512, 512, 3)

### Evaluate the model

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

Our simple CNN has achieved a test accuracy of over 70%. Not bad for a few lines of code! For another CNN style, see an example using the Keras subclassing API and a `tf.GradientTape` [here](https://www.tensorflow.org/tutorials/quickstart/advanced).